In [1]:
from time import time
import matplotlib.pyplot as plt
import sys

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import random_projection
from sklearn.utils.fixes import loguniform
from tqdm import tqdm
import numpy as np
import pickle

In [2]:
def get_data():
    from datamodule import ImageWeightsModule
    import torch
    all_data_loader = ImageWeightsModule("lora_dataset_creator/lora_dataset", 1, augment_training=False, val_split=0).train_dataloader()
    X = None
    Y = None
    for x, y in tqdm(all_data_loader):
        if X is None:
            X = x
        X = torch.cat((X, x), dim=0)
        if Y is None:
            Y = y
        Y = torch.cat((Y, y), dim=0)
    return X, np.array(Y)

X, Y = get_data()

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


filter_files: 335 approved - 0 rejected


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 335/335 [00:49<00:00,  6.74it/s]


In [3]:
scaler = StandardScaler()
Y_scaled = scaler.fit_transform(Y)
Y_scaled = Y
Y_scaled_val = Y_scaled[:10, :]
Y_scaled = Y_scaled[10:, :]
print(Y_scaled.shape)

(326, 509248)


In [4]:
@np.printoptions(suppress=True)
def test_pca(x):
    t0 = time()
    z = pca.transform(x)
    print("transform done in %0.3fs" % (time() - t0))
    print(f"out = {z.shape}")
    t0 = time()
    x_hat = pca.inverse_transform(z)
    print("inverse_transform done in %0.3fs" % (time() - t0))
    print(f"loss = {abs(x_hat - x).mean()}")

In [5]:
n_components = 100
t0 = time()
print(f"in = {Y_scaled.shape}")
pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True, random_state=5).fit(Y_scaled)
print("done in %0.3fs" % (time() - t0))

in = (326, 509248)
done in 22.227s


In [6]:
test_pca(Y_scaled)
test_pca(Y_scaled_val)

transform done in 1.527s
out = (326, 100)
inverse_transform done in 1.137s
loss = 0.02307949960231781
transform done in 0.278s
out = (10, 100)
inverse_transform done in 0.178s
loss = 0.03913899138569832


In [7]:
eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_20020/2267776780.py:1 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_20020/2267776780.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'eigenfaces' is not defined

In [ ]:
print("Fitting the classifier to the training set")
t0 = time()
param_grid = {
    "C": loguniform(1e3, 1e5),
    "gamma": loguniform(1e-4, 1e-1),
}
clf = RandomizedSearchCV(
    SVC(kernel="rbf", class_weight="balanced"), param_grid, n_iter=10
)
clf = clf.fit(X_train_pca, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

In [ ]:
print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred, target_names=target_names))
ConfusionMatrixDisplay.from_estimator(
    clf, X_test_pca, y_test, display_labels=target_names, xticks_rotation="vertical"
)
plt.tight_layout()
plt.show()